In [ ]:
def evaluate_reconstruction(adj_matrix, opt_precision_mat, threshold=1e-5):
    """
    Evaluate the accuracy of the reconstructed adjacency matrix.

    Parameters
    ----------
    adj_matrix : array-like, shape (p, p)
        The original adjacency matrix.
    opt_precision_mat : array-like, shape (p, p)
        The optimized precision matrix.
    threshold : float, optional
        The threshold for considering an edge in the precision matrix. Default is 1e-5.

    Returns
    -------
    metrics : dict
        Dictionary containing precision, recall, f1_score, and jaccard_similarity.
    """
    # Convert the optimized precision matrix to binary form
    reconstructed_adj = (np.abs(opt_precision_mat) > threshold).astype(int)
    np.fill_diagonal(reconstructed_adj, 0)

    # True positives, false positives, etc.
    tp = np.sum((reconstructed_adj == 1) & (adj_matrix == 1))
    fp = np.sum((reconstructed_adj == 1) & (adj_matrix == 0))
    fn = np.sum((reconstructed_adj == 0) & (adj_matrix == 1))
    tn = np.sum((reconstructed_adj == 0) & (adj_matrix == 0))

    # Precision, recall, F1 score
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Jaccard similarity
    jaccard_similarity = tp / (tp + fp + fn)

    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'jaccard_similarity': jaccard_similarity,
        'accuracy': accuracy
    }

    return metrics

In [ ]:
num_runs = 30
p = 20
n = 500
b = int(n / 2)
Q = 100
lambda_granularity = 100
lambda_range_np = np.linspace(0.01, 0.4, lambda_granularity)
lambda_range_wp = np.linspace(0.01, 0.4, lambda_granularity)

all_mean_metrics = {}
successful_runs_per_lambda = {}
# sample run
for lambda_idx in tqdm(range(len(lambda_range_np))):
    mean_f1_score = 0
    mean_precision = 0
    mean_recall = 0
    mean_accuracy = 0
    mean_jaccard_similarity = 0
    counter = 0
    for run in range(num_runs):
        lambda_np, lambda_wp, opt_precision_mat, adj_matrix = synthetic_run(lambda_range_np[lambda_idx], lambda_range_wp[lambda_idx], p=p, n=n, b=b, Q=Q)

        # sample evaluation
        if np.any(opt_precision_mat != 0):
            metrics = evaluate_reconstruction(adj_matrix, opt_precision_mat)
            mean_f1_score += metrics['f1_score']
            mean_precision += metrics['precision']
            mean_recall += metrics['recall']
            mean_accuracy += metrics['accuracy']
            mean_jaccard_similarity += metrics['jaccard_similarity']


            counter += 1
    if counter == 0:
        counter = 1
    mean_f1_score /= counter
    mean_precision /= counter
    mean_recall /= counter
    mean_accuracy /= counter
    mean_jaccard_similarity /= counter
    all_mean_metrics[str(f'{lambda_range_np[lambda_idx]} / {lambda_range_wp[lambda_idx]}')] = {'f1_score': mean_f1_score,
                                                         'precision': mean_precision, 'recall': mean_recall, 
                                                         'jaccard_similarity': mean_jaccard_similarity, 'accuracy': mean_accuracy}
    successful_runs_per_lambda[str(f'{lambda_range_np[lambda_idx]} / {lambda_range_wp[lambda_idx]}')] = counter


for key in all_mean_metrics.keys():
    print(key)
    print(all_mean_metrics[key])
    print('\n')
print('number of successful runs per lambda:')
print(successful_runs_per_lambda)